<a href="https://colab.research.google.com/github/Jira-saki/Data_Analysis_using_python/blob/main/Data_Cleansing_with_Spark_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleansing with Spark

## Dataset
https://archive.ics.uci.edu/ml/datasets/Online+Retail
### Data Dictionary

This is a transactional data set which contains all the transactions occurring between 01/12/2018 and 09/12/2019 for a UK-based and registered non-store online retail.

The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.


## Pyspark installation 

In [ ]:
# Install Spark for Google Colab
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0

In [ ]:
# Setting Spark enviroment variable 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
# Installing pyspark 
!pip install pyspark==3.1.2

#### Working with Spark

In [ ]:
# Check Google Colab Core Server.
!cat /proc/cpuinfo

In [ ]:
# Building Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "legacy") # code for Spark3 to process like Spark2 (Preventing an Error when using function)

In [ ]:
# Get Python version
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)

In [ ]:
# Get Spark version
spark.version

'3.1.2'

#### Mounting to Google Drive


In [ ]:
# Mount Google colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data

`spark.read.csv` to read CSV file.

Arguments to Spark
Header = True (First sentence of CSV is Header)
Inferschema = True << (Spark will assume schema type of each column. If "False", columns will be assumed as string.

In [ ]:
dt = spark.read.csv('/content/drive/MyDrive/Online Retail WS2 (1).csv', header = True, inferSchema = True, )

### Data Profiling

Data Profiling is a process of analysing summary of the data.

Example: max, min, average, sum, how many missing values etc.

#### Data

> Columns
- InvoiceNo
- StockCode
- Description
- Quantity
- InvoiceDate
- UnitPrice
- CustomerID
- Country

In [ ]:
dt

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [ ]:
dt.show()

In [ ]:
dt.show(100)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1/12/2018 08:26|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1/12/2018 08:26|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1/12/2018 08:26|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1/12/2018 08:26|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1/12/2018 08:26|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1/12/2018 08:26|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1/12/2018 08:

In [ ]:
# Show Schema
dt.dtypes

[('InvoiceNo', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('UnitPrice', 'double'),
 ('CustomerID', 'double'),
 ('Country', 'string')]

In [ ]:
# Show Schema with nullable
dt.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
# Count columns
print((dt.count(), len(dt.columns)))

(541909, 8)


In [ ]:
# Statistic Summary
dt.describe().show()

+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|    InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|         541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|           null|4.611113626082972|15287.690570239585|       null|
| stddev|13428.417280805186| 16799.73762842771|                null|218.08115785023477|           null|96.75985306117933|1713.6003033215918|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -8

In [ ]:
# Statistic Summary
dt.summary().show()

+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|    InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|         541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|           null|4.611113626082972|15287.690570239585|       null|
| stddev|13428.417280805186| 16799.73762842771|                null|218.08115785023477|           null|96.75985306117933|1713.6003033215918|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -8

In [ ]:
#Statistic Summary for specific column ท
dt.select("Quantity", "UnitPrice").describe().show()

+-------+------------------+-----------------+
|summary|          Quantity|        UnitPrice|
+-------+------------------+-----------------+
|  count|            541909|           541909|
|   mean|  9.55224954743324|4.611113626082972|
| stddev|218.08115785023477|96.75985306117933|
|    min|            -80995|        -11062.06|
|    max|             80995|          38970.0|
+-------+------------------+-----------------+



### Median of Quantity

In [ ]:
#dt.select("Quantity").summary().show()
dt.select("Quantity").summary().collect()[5]

Row(summary='50%', Quantity='3')

## EDA - Exploratory Data Analysis

## Non-Graphical EDA

In [2]:
# Select text-based information
dt.where(dt['Quantity'] < 0).show()

NameError: ignored

### Assumption 
1. Find Quantity of 50 - 120
2. Find UnitPrice of 0.1 - 0.5
3. Quantity 50 - 120 ,UnitPrice  0.1 - 0.5

In [3]:
#  Quantity 50 - 120
dt.where( (dt['Quantity'] > 50) & (dt['Quantity'] < 120) ).show()

NameError: ignored

In [ ]:
#UnitPrice 0.1 - 0.5
dt.where( (dt['Unitprice'] >= 0.1) & (dt['Unitprice'] <= 0.5) ).show()

In [ ]:
# Quantity 50 - 120 and UnitPrice 0.1 - 0.5
dt.where( dt["Quantity"]. between(50,120) & dt ["Unitprice"]. between(0.1,0.5) ).show()

### Graphical EDA


Importing packages seaborn, matplotlib, pandas for graph visualisation.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Convert Spark Dataframe to Pandas Dataframe
dt_pd = dt.toPandas()

In [ ]:
dt_pd.head()

In [ ]:
# Select 500 rows for data visualise.
dt_pd_subset = dt_pd[0:500]

In [ ]:
# Boxplot
sns.boxplot(dt_pd_subset['UnitPrice'])

In [ ]:
# Histogram *distplot= distribution plot
sns.distplot(dt_pd_subset['UnitPrice']) 
plt.show()

In [ ]:
# Scatterplot to see the contrast.
dt_pd_subset.plot.scatter('UnitPrice', 'Quantity')

#### Interactive chart

In [ ]:
# Plotly - interactive chart
import plotly.express as px
fig = px.scatter(dt_pd_subset, 'UnitPrice', 'Quantity')
fig.show()

### Type Conversion

Convert `InvoiceDate` from string -> date

In [ ]:
# Show top 5 rows
dt.show(5)

In [ ]:
# Show Schema
dt.printSchema()

Is the date DD/MM/YYYY or MM/DD/YYYY? Let's find out



In [ ]:
# Show unique Invoice Date
dt.select("InvoiceDate").distinct().show()

In [ ]:
# Convert string to date  with new column using  df.withColumn()
from pyspark.sql import functions as f

# dt_temp = dt.withColumn('InvoiceDateTime', functions.to_date(
#     functions.unix_timestamp('InvoiceDate', 'dd/MM/yyyy HH:mm').cast('timestamp')
# ))

dt_temp = dt.withColumn('InvoiceDateTime', 
    f.unix_timestamp('InvoiceDate', 'dd/MM/yyyy HH:mm').cast('timestamp')
)
dt_temp.show()

In [ ]:
dt_temp.printSchema()

In [ ]:
dt_final = dt_temp.drop('InvoiceDate')
dt_final.show()

In [ ]:
dt_final.printSchema()

## Data Cleansing with Spark

### Anomalies Check

#### Syntactical Anomalies
**Lexical errors** (wrong typing)

In [ ]:
# Check country distinct values.
dt_final.select("Country").distinct().show()

In [ ]:
dt_final.where(dt_final['Country'] == 'EIREs').show()

In [ ]:
# Modify an error name
from pyspark.sql.functions import when

dt_temp_eire = dt_final.withColumn("CountryUpdate", when(dt_final['Country'] == 'EIREs', 'EIRE').otherwise(dt_final['Country']))

In [ ]:
# Check the result
dt_temp_eire.select("CountryUpdate").distinct().show()

In [ ]:
# Create final Dataframe
dt_final_eire = dt_temp_eire.drop("Country").withColumnRenamed('CountryUpdate', 'Country')

In [ ]:
dt_final_eire.show()

#### Semantic Anomalies

**Integrity constraints**

In [ ]:
dt_final_eire.select("Stockcode").show(100)

In [ ]:
dt_final_eire.count()

In [ ]:
dt_final_eire.filter(dt_final_eire["Stockcode"].rlike("^[0-9]{5}$")).count()

In [ ]:
# Check data
dt_final_eire.filter(dt_final_eire["Stockcode"].rlike("^[0-9]{5}$")).show(5)

In [ ]:
# check incorrect data
dt_correct_stockcode = dt_final_eire.filter(dt_final_eire["Stockcode"].rlike("^[0-9]{5}$"))
dt_incorrect_stockcode = dt_final_eire.subtract(dt_correct_stockcode)

dt_incorrect_stockcode.show(10)

> Finding incorrect Pattern of Stock Code

In [ ]:
# remove the last string from stock code #  ,r'[A-Z]', '')  find A-Z replace with " " (blank)
from pyspark.sql.functions import regexp_replace

dt_temp_stockcode = dt_final_eire.withColumn("StockcodeUpdate", regexp_replace(dt_final_eire['Stockcode'], r'[A-Z]', ''))

In [ ]:
# Check the result
dt_temp_stockcode.show()

In [ ]:
# Create final Dataframe
dt_final_stockcode = dt_temp_stockcode.drop("Stockcode").withColumnRenamed('StockcodeUpdate', 'StockCode')

In [ ]:
dt_final_stockcode.show(4)

#### Missing values

Check Missing Values

In [ ]:
# Check missing values in each column
# (*[sum(col(c).isNull().cast("int")).alias(c)  find sum() of column"c" Count null value with True,False  and change to "int" 
from pyspark.sql.functions import col,sum

dt_final_stockcode.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in dt_final_stockcode.columns]).show()

In [ ]:
# Check which row of 'Description' = null.

dt_final_stockcode.where( dt_final_stockcode['Description'].isNull() ).show()

In [ ]:
# Check which row of "customerID" = null.

dt_final_stockcode.where( dt_final_stockcode['customerID'].isNull() ).show()

### Use Case:
Data Analyst would like to replace "Customer ID"'s NULL to -1

In [ ]:
# Write code here
dt_customer_notnull = dt_final_stockcode.withColumn("CustomerIDUpdate", when(dt_final_stockcode['customerID'].isNull(),  -1).otherwise(dt_final_stockcode['customerID']))
dt_customer_notnull.show()

+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+----------------+
|InvoiceNo|         Description|Quantity|UnitPrice|CustomerID|    InvoiceDateTime|       Country|StockCode|CustomerIDUpdate|
+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+----------------+
|   536365|WHITE HANGING HEA...|       6|     2.55|   17850.0|2018-12-01 08:26:00|United Kingdom|    85123|         17850.0|
|   536365| WHITE METAL LANTERN|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    71053|         17850.0|
|   536365|CREAM CUPID HEART...|       8|     2.75|   17850.0|2018-12-01 08:26:00|United Kingdom|    84406|         17850.0|
|   536365|KNITTED UNION FLA...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|         17850.0|
|   536365|RED WOOLLY HOTTIE...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|         17850.0|


### Clean data with Spark SQL


Value of `unitPrice` , `Quantity` is Corrected (> 0)

In [ ]:
dt_final_stockcode.createOrReplaceTempView("sales")
dt_sql = spark.sql("SELECT * FROM sales")
dt_sql.show()

+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+
|InvoiceNo|         Description|Quantity|UnitPrice|CustomerID|    InvoiceDateTime|       Country|StockCode|
+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+
|   536365|WHITE HANGING HEA...|       6|     2.55|   17850.0|2018-12-01 08:26:00|United Kingdom|    85123|
|   536365| WHITE METAL LANTERN|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    71053|
|   536365|CREAM CUPID HEART...|       8|     2.75|   17850.0|2018-12-01 08:26:00|United Kingdom|    84406|
|   536365|KNITTED UNION FLA...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|
|   536365|RED WOOLLY HOTTIE...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|
|   536365|SET 7 BABUSHKA NE...|       2|     7.65|   17850.0|2018-12-01 08:26:00|United Kingdom|    22752|
|   536365|GLASS STAR FROSTE

In [ ]:
dt_sql_count = spark.sql("SELECT count(*) as cnt_row FROM sales")
dt_sql_count.show()

+-------+
|cnt_row|
+-------+
| 541909|
+-------+



In [ ]:
dt_sql_count = spark.sql("SELECT count(*) as cnt_row, country FROM sales GROUP BY Country ORDER BY cnt_row DESC")
dt_sql_count.show()

+-------+---------------+
|cnt_row|        country|
+-------+---------------+
| 495478| United Kingdom|
|   9495|        Germany|
|   8557|         France|
|   8196|           EIRE|
|   2533|          Spain|
|   2371|    Netherlands|
|   2069|        Belgium|
|   2002|    Switzerland|
|   1519|       Portugal|
|   1259|      Australia|
|   1086|         Norway|
|    803|          Italy|
|    758|Channel Islands|
|    695|        Finland|
|    622|         Cyprus|
|    462|         Sweden|
|    446|    Unspecified|
|    401|        Austria|
|    389|        Denmark|
|    358|          Japan|
+-------+---------------+
only showing top 20 rows



In [ ]:
dt_sql_valid_price = spark.sql("SELECT count(*) as cnt_row FROM sales WHERE UnitPrice > 0 AND Quantity > 0")
dt_sql_valid_price.show()

+-------+
|cnt_row|
+-------+
| 530104|
+-------+



In [ ]:
dt_sql_valid_price = spark.sql("SELECT * FROM sales WHERE UnitPrice > 0 AND Quantity > 0")
dt_sql_valid_price.show()

+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+
|InvoiceNo|         Description|Quantity|UnitPrice|CustomerID|    InvoiceDateTime|       Country|StockCode|
+---------+--------------------+--------+---------+----------+-------------------+--------------+---------+
|   536365|WHITE HANGING HEA...|       6|     2.55|   17850.0|2018-12-01 08:26:00|United Kingdom|    85123|
|   536365| WHITE METAL LANTERN|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    71053|
|   536365|CREAM CUPID HEART...|       8|     2.75|   17850.0|2018-12-01 08:26:00|United Kingdom|    84406|
|   536365|KNITTED UNION FLA...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|
|   536365|RED WOOLLY HOTTIE...|       6|     3.39|   17850.0|2018-12-01 08:26:00|United Kingdom|    84029|
|   536365|SET 7 BABUSHKA NE...|       2|     7.65|   17850.0|2018-12-01 08:26:00|United Kingdom|    22752|
|   536365|GLASS STAR FROSTE

### TODO: 
1.  Country USA which InvoiceDateTime from 2010-12-01  and UnitPrice > 3.5 
2.  Country France which InvoiceDateTime from 2010-12-05 and UnitPrice  > 5.5 and Description with "Box"

In [ ]:
# TODO: Country USA which InvoiceDateTime from 2010-12-01 and UnitPrice > 3.5
dt_sql_valid_price = spark.sql("""
SELECT * FROM sales 
  WHERE UnitPrice > 3.5 
  AND Country = 'USA' 
  AND InvoiceDateTime >= '2010-12-01'
""")
dt_sql_valid_price.show()

+---------+--------------------+--------+---------+----------+-------------------+-------+---------+
|InvoiceNo|         Description|Quantity|UnitPrice|CustomerID|    InvoiceDateTime|Country|StockCode|
+---------+--------------------+--------+---------+----------+-------------------+-------+---------+
|   550644|SET OF 6 SPICE TI...|       7|     3.95|   12733.0|2019-04-19 16:19:00|    USA|    22722|
|   550644|SET OF 3 CAKE TIN...|       1|     4.95|   12733.0|2019-04-19 16:19:00|    USA|    22720|
|   550644|PINK HAPPY BIRTHD...|       4|     5.45|   12733.0|2019-04-19 16:19:00|    USA|    47590|
|   550644|COTTON APRON PANT...|       5|     4.95|   12733.0|2019-04-19 16:19:00|    USA|    22990|
|   550644|REGENCY CAKESTAND...|       6|    12.75|   12733.0|2019-04-19 16:19:00|    USA|    22423|
|   550644|TEA TIME TEA SET ...|       5|     6.95|   12733.0|2019-04-19 16:19:00|    USA|    37501|
|   550644|TEA TIME TEAPOT I...|       4|     4.95|   12733.0|2019-04-19 16:19:00|    USA| 

In [ ]:
# Country France which InvoiceDateTime from 2010-12-05 and UnitPrice > 5.5 and Description with "Box"
dt_sql_valid_price = spark.sql("""
SELECT *  
  FROM sales 
  WHERE UnitPrice >= 5.5 
  AND InvoiceDateTime >= '2010-12-05' 
  And Country = 'France'
  AND LOWER(Description) LIKE '%box%'
""").show()

+---------+--------------------+--------+---------+----------+-------------------+-------+---------+
|InvoiceNo|         Description|Quantity|UnitPrice|CustomerID|    InvoiceDateTime|Country|StockCode|
+---------+--------------------+--------+---------+----------+-------------------+-------+---------+
|   537065|ROSE COTTAGE KEEP...|       8|      8.5|   12567.0|2018-12-05 11:57:00| France|    22968|
|   537065|  TOOL BOX SOFT TOY |       6|     8.95|   12567.0|2018-12-05 11:57:00| France|    21055|
|   537463|BOOM BOX SPEAKER ...|       4|     5.95|   12681.0|2018-12-07 10:08:00| France|    21064|
|   537897|SET 7 BABUSHKA NE...|       2|      8.5|   12683.0|2018-12-09 10:37:00| France|    22752|
|   539607|BOOM BOX SPEAKER ...|       4|     5.95|   12681.0|2018-12-20 14:10:00| France|    21065|
|   540178|BOOM BOX SPEAKER ...|       2|     5.95|   12681.0|2019-01-05 12:42:00| France|    21064|
|   540351|  TOOL BOX SOFT TOY |       1|     8.95|   12735.0|2019-01-06 14:26:00| France| 

## Save cleaned data to CSV

In [ ]:
# Write as partitioned files (use multiple workers)
dt_sql_valid_price.write.csv('Cleaned_Data_Now_Final.csv')

In [ ]:
# Read CSV file 1
part1 = spark.read.csv('/content/Cleaned_Data_Now_Final.csv', header = True, inferSchema = True, )
part1.count()

530102

In [ ]:
# Read CSV file 2
part2 = spark.read.csv('/content/Cleaned_Data_Now_Final_Single.csv', header = True, inferSchema = True, )
part2.count()

530103

In [ ]:
# Write as 1 file (use single worker)
dt_sql_valid_price.coalesce(1).write.csv('Cleaned_Data_Now_Final_Single.csv')

In [ ]:
dt_sql_valid_price.count()

530104

In [ ]:
# Read CSV file 
dt = spark.read.csv('/content/Cleaned_Data_Now_Final.csv/part-*.csv', header = True, inferSchema = True, )


In [ ]:
dt.count()

530102